In [1]:
import numpy as np
import cvxpy as cp
from qutip import Qobj, tensor, ptrace, qeye, num, ket2dm
from numpy.polynomial.hermite import hermgauss, Hermite
from math import factorial
from itertools import zip_longest

ModuleNotFoundError: No module named 'cvxpy'

In [ ]:
def cv_state_S26(r, eta1, eta2, max_p):
    
    t = np.tanh(r)
    sqrt_eta1, sqrt_1_e1 = np.sqrt(eta1), np.sqrt(1-eta1)
    d = max_p+1
    amps = {}  # map (nA,nB,nC)->complex amplitude

    fac = factorial
    for p in range(max_p+1):
        pref_p = (t**p) / ((2**p) * np.cosh(r))
        for k1 in range(p+1):
            f_k1 = (2*eta1-1)**k1
            for k2 in range(k1+1):
                f_bs1 = (4*sqrt_eta1*sqrt_1_e1)**(p-k1)
                num1 = fac(p + 2*k2 - k1) * fac(p - (2*k2 - k1))
                den1 = fac(p-k1)*fac(k1-k2)*fac(k2)
                sqrt1 = np.sqrt(num1/den1)

                nA = p + (2*k2 - k1)
                nB1= p - (2*k2 - k1)

                for k3 in range(nB1+1):
                    # binomial factor & second-BS sqrt
                    c = fac(nB1) / (fac(k3)*fac(nB1-k3))
                    sqrt2 = np.sqrt(c * eta2**k3 * (1-eta2)**(nB1-k3))

                    sign = (-1)**(p - (3*k2 + k3))
                    nB, nC = k3, nB1-k3

                    amp = pref_p * f_bs1 * f_k1 * sqrt1 * sqrt2 * sign
                    amps[(nA,nB,nC)] = amps.get((nA,nB,nC),0) + amp

    vec = []
    for nA in range(d):
      for nB in range(d):
        for nC in range(d):
          vec.append(amps.get((nA,nB,nC),0.0))
    psi = Qobj(np.array(vec), dims=[[d,d,d],[1,1,1]]).unit()
    return psi

In [ ]:
def make_homodyne_povm(N_bins, T, cutoff, Nq=200):
    d = cutoff + 1

    # Gauss–Hermite nodes/weights
    x, w = hermgauss(Nq)

    # Build Φ_n(x_i) = norm_n * H_n(x_i) 
    Phi = np.empty((d, Nq))
    for n in range(d):
        Hn   = Hermite.basis(n)  # physicists' H_n
        norm = 1.0 / (np.pi**0.25 * np.sqrt(2.0**n * factorial(n)))
        Phi[n, :] = norm * Hn(x)

    # Periodic wrap and bin index (each node lands in exactly one bin)
    tri    = T / N_bins
    xmod = x - np.floor(x / T) * T          # in [0, T)
    idx  = np.floor(xmod / tri).astype(int)
    idx  = np.clip(idx, 0, N_bins - 1)

    Ms = []
    for b in range(N_bins):
        mask = (idx == b).astype(float)     # shape (Nq,)
        W    = w * mask
        mat  = (Phi * W) @ Phi.T            # (d,Nq)*(Nq,) -> (d,Nq); @ Phi.T -> (d,d)
        Ms.append(Qobj(mat, dims=[[d],[d]]))

    # Completeness: Σ_b M_b ≈ I
    I = sum(M.full() for M in Ms)
    assert np.allclose(I, np.eye(d), atol=1e-10), "Σ_b M_b ≠ I; try larger Nq."
    return Ms

In [ ]:
def certify_randomness_ns(psi, Mbases, Mcases):
    d = psi.dims[0][0]           # local Hilbert dim (should be cutoff+1)
    rho = ket2dm(psi)            # make it a density operator
    nB = len(Mbases['x'])   
    nC = len(Mcases['x'])  

    # 1) assemblage
    sigma_obs = {}
    for y in ['x','p']:
        for z in ['x','p']:
            for b in range(nB):
                for c in range(nC):
                     Mb, Mc = Mbases[y][b], Mcases[z][c]
                     op = tensor(qeye(d), Mb, Mc)              # I ⊗ Mb ⊗ Mc
                     sigma_obs[(b,c,y,z)] = ptrace(op * rho, [0])  # on A

    # 2) variables
    sigma = {}
    for e in range(nB):
        for e2 in range(nC):
            for y in ['x','p']:
                for z in ['x','p']:
                    for b in range(nB):
                        for c in range(nC):
                            sigma[(e,e2,b,c,y,z)] = cp.Variable((d,d), hermitian=True)

    constraints = []

    # 3) equality to observed assemblage
    for (b,c,y,z), obs in sigma_obs.items():
        lhs = sum(sigma[(e,e2,b,c,y,z)] for e in range(nB) for e2 in range(nC))
        constraints.append(lhs == cp.Constant(obs.full()))

    # 4) no-signaling per (B6)
    for e in range(nB):
        for e2 in range(nC):
        # C marginal independent of y
            for c in range(nC):
                  for z in ['x','p']:
                        lhs1 = sum(sigma[(e,e2,b,c,'x',z)] for b in range(nB))
                        lhs2 = sum(sigma[(e,e2,b,c,'p',z)] for b in range(nB))
                        constraints.append(lhs1 == lhs2)
        # B marginal independent of z
            for b in range(nB):
                  for y in ['x','p']:
                        lhs1 = sum(sigma[(e,e2,b,c,y,'x')] for c in range(nC))
                        lhs2 = sum(sigma[(e,e2,b,c,y,'p')] for c in range(nC))
                        constraints.append(lhs1 == lhs2)

    # 5) PSD
    for var in sigma.values():
        constraints.append(var >> 0)

    # 6) objective
    obj_expr = cp.sum([
        cp.real(cp.trace(sigma[(e,e2,e,e2,'x','x')]))
        for e in range(nB) for e2 in range(nC)
   ])
    prob = cp.Problem(cp.Maximize(obj_expr), constraints)
    result = prob.solve(solver=cp.SCS, verbose=False, max_iters=20000, eps=1e-7)

    P_g  = float(prob.value)                   # optimal guessing prob
    P_g  = float(np.clip(P_g, 0.0, 1.0))       # guard against tiny numerical drift
    Hmin = 0.0 if P_g <= 0 else max(0.0, -np.log2(P_g))
    
    return Hmin, P_g


In [ ]:
if __name__ == "__main__":
    # Fig. 4(c) parameters
    r      = 0.345
    eta1   = 0.5
    p_max  = 1         
    cutoff = 1        
    d      = cutoff + 1
    N_bins = 4
    T_B_vals = np.linspace(2, 10, 9)   # search Bob’s Tx
    T_C_vals = np.linspace(2, 10, 9)   # search Charlie’s Tx
    Nq     = 160                       

    R_pi2 = (-1j * (np.pi/2) * num(d)).expm()

    Hmins = []
    for eta2 in np.linspace(0, 1, 11):
        best_H = 0.0

        psi = cv_state_S26(r, eta1, eta2, p_max)

        for TxB in T_B_vals:
            TpB = 2*np.pi*N_bins / TxB

            # Bob POVMs
            Mb_x = make_homodyne_povm(N_bins, TxB, cutoff, Nq=Nq)
            Mb_p_xform = make_homodyne_povm(N_bins, TpB, cutoff, Nq=Nq)
            Mb_p = [R_pi2.dag() * M * R_pi2 for M in Mb_p_xform]

            for TxC in T_C_vals:
                TpC = 2*np.pi*N_bins / TxC

                # Charlie POVMs
                Mc_x = make_homodyne_povm(N_bins, TxC, cutoff, Nq=Nq)
                Mc_p_xform = make_homodyne_povm(N_bins, TpC, cutoff, Nq=Nq)
                Mc_p = [R_pi2.dag() * M * R_pi2 for M in Mc_p_xform]

                Hmin, Pg = certify_randomness_ns(
                    psi,
                    {'x': Mb_x, 'p': Mb_p},
                    {'x': Mc_x, 'p': Mc_p},
                )
                best_H = max(best_H, Hmin)

        Hmins.append(best_H)

In [ ]:
print(Hmins)